#Getting meaning from a text

###Using TextBlob, NLTK and Gensim

In [36]:
import string

from nltk.stem.wordnet import WordNetLemmatizer

from gensim import models
from gensim.corpora import Dictionary

In [51]:
## Example text:
zen = ["Beautiful is better than ugly. Explicit is better than implicit.", 
        "Simple is better than complex. Complex is better than complicated.",
        "Flat is better than nested. Sparse is better than dense.",
        "Readability counts. Special cases aren't special enough to break the rules.",
        "Although practicality beats purity. Errors should never pass silently.",
        "Unless explicitly silenced. In the face of ambiguity, refuse the temptation to guess." ,
        "There should be one-- and preferably only one --obvious way to do it.",
        "Although that way may not be obvious at first unless you're Dutch.",
        "Now is better than never. Although never is often better than *right* now.",
        "If the implementation is hard to explain, it's a bad idea.",
        "If the implementation is easy to explain, it may be a good idea."
        "Namespaces are one honking great idea -- let's do more of those!"]

In [84]:
## TOKENISATION:
## Breaking down a text in meaningful elements
texts = [text.lower().replace('\n', ' ').split(' ') for text in zen]
print texts

stop_words = ['a', 'an', 'and', 'are', 'as', 'at', 'be', 'by', 'for'
             'from', 'has', 'he', 'if', 'in', 'is', 'it', 'its', 'it\'s', 'of', 'on', 
             'than', 'that', 'the', 'to', 'was', 'were', 'will', 'with']

docs = [[filter(lambda x:x not in string.punctuation, i) for i in txt if i != '' and i not in stop_words] 
        for txt in texts]
print docs

[['beautiful', 'better', 'ugly', 'explicit', 'better', 'implicit'], ['simple', 'better', 'complex', 'complex', 'better', 'complicated'], ['flat', 'better', 'nested', 'sparse', 'better', 'dense'], ['readability', 'counts', 'special', 'cases', 'arent', 'special', 'enough', 'break', 'rules'], ['although', 'practicality', 'beats', 'purity', 'errors', 'should', 'never', 'pass', 'silently'], ['unless', 'explicitly', 'silenced', 'face', 'ambiguity', 'refuse', 'temptation', 'guess'], ['there', 'should', 'one', 'preferably', 'only', 'one', 'obvious', 'way', 'do', 'it'], ['although', 'way', 'may', 'not', 'obvious', 'first', 'unless', 'youre', 'dutch'], ['now', 'better', 'never', 'although', 'never', 'often', 'better', 'right', 'now'], ['implementation', 'hard', 'explain', 'its', 'bad', 'idea'], ['implementation', 'easy', 'explain', 'may', 'good', 'ideanamespaces', 'one', 'honking', 'great', 'idea', '', 'lets', 'do', 'more', 'those']]
[['beautiful', 'better', 'ugly', 'explicit', 'better', 'implic

In [85]:
## LEMMATISATION:
## Grouping together the different forms of a word
lmtzr = WordNetLemmatizer()
lemm = [[lmtzr.lemmatize(word) for word in data] for data in docs]
print lemm

[['beautiful', 'better', 'ugly', 'explicit', 'better', 'implicit'], ['simple', 'better', 'complex', 'complex', 'better', 'complicated'], ['flat', 'better', 'nested', 'sparse', 'better', 'dense'], ['readability', u'count', 'special', u'case', 'arent', 'special', 'enough', 'break', u'rule'], ['although', 'practicality', u'beat', 'purity', u'error', 'should', 'never', u'pas', 'silently'], ['unless', 'explicitly', 'silenced', 'face', 'ambiguity', 'refuse', 'temptation', 'guess'], ['there', 'should', 'one', 'preferably', 'only', 'one', 'obvious', 'way', 'do', 'it'], ['although', 'way', 'may', 'not', 'obvious', 'first', 'unless', 'youre', 'dutch'], ['now', 'better', 'never', 'although', 'never', 'often', 'better', 'right', 'now'], ['implementation', 'hard', 'explain', 'bad', 'idea'], ['implementation', 'easy', 'explain', 'may', 'good', 'ideanamespaces', 'one', 'honking', 'great', 'idea', '', u'let', 'do', 'more', 'those']]


In [86]:
## Create bag of words from dictionnary:
####Note: compare doc2bow and word2vec
dictionary = Dictionary(lemm)
dictionary.save('text.dict')

## Term frequency–inverse document frequency (TF-IDF)
## Method to reflect how important a word is to a document in a collection.
## The inverse document frequency measures whether the term is common or rare across all documents.
bow = [dictionary.doc2bow(l) for l in lemm] # Calculates inverse document counts for all terms
tfidf = models.TfidfModel(bow)              # Transforms the count representation into the Tfidf space
corpus_tfidf = tfidf[bow]

In [87]:
## Build the LSI model
## Method to uses a mathematical technique called singular value decomposition (SVD) 
## to identify patterns in the relationships between the terms and concepts contained 
## in an unstructured collection of text.
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=3)
corpus_lsi = lsi[corpus_tfidf]

In [88]:
for i in range(lsi.num_topics):
    print lsi.show_topic(i)

[(-0.54099716727821068, u'better'), (-0.27853037146820386, u'complex'), (-0.27731795093733819, u'now'), (-0.25384405967038415, u'never'), (-0.18163236900752361, u'dense'), (-0.18163236900752347, u'nested'), (-0.18163236900752339, u'sparse'), (-0.18163236900752333, u'ugly'), (-0.1816323690075233, u'flat'), (-0.18163236900752308, u'implicit')]
[(-0.30794775806820202, u'one'), (-0.25292223047307155, u'implementation'), (-0.25292223047307155, u'idea'), (-0.25292223047307155, u'explain'), (-0.2077880505088438, u'do'), (-0.20437009294542663, u'bad'), (-0.20437009294542663, u'hard'), (0.19505419477989211, u'better'), (-0.18973512044056631, u'may'), (-0.18226648505043938, u'obvious')]
[(-0.24330841921959806, u'bad'), (-0.243308419219598, u'hard'), (-0.23373524267885931, u'explain'), (-0.23373524267885928, u'implementation'), (-0.23373524267885928, u'idea'), (0.20928347389567789, u'never'), (0.20727316973039564, u'although'), (0.17738986606596713, u'should'), (0.17342762797286618, u'obvious'), 

In [96]:
list_topics = [] 
for i in range(lsi.num_topics):
    list_topics.extend(lsi.show_topic(i))
    
topics = [i[1] for i in list_topics]
print topics

[u'better', u'complex', u'now', u'never', u'dense', u'nested', u'sparse', u'ugly', u'flat', u'implicit', u'one', u'implementation', u'idea', u'explain', u'do', u'bad', u'hard', u'better', u'may', u'obvious', u'bad', u'hard', u'explain', u'implementation', u'idea', u'never', u'although', u'should', u'obvious', u'way']
